# Running Inference

In [ ]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation


/home/guru/git_reps/transformer_pytorch/myenv_jetson/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config, f"10")
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Using device cuda
Max length of source sentence: 466
Max length of target sentence: 479


<All keys matched successfully>

In [3]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

stty: 'standard input': Inappropriate ioctl for device


--------------------------------------------------------------------------------
    SOURCE: "Messire," said Liénarde.
    TARGET: »Herr,« sagte Liénarde.
 PREDICTED: » Herr ,« sagte Liénarde .
--------------------------------------------------------------------------------
    SOURCE: He picked up a clean pine shingle that lay in the moon-light, took a little fragment of "red keel" out of his pocket, got the moon on his work, and painfully scrawled these lines, emphasizing each slow down-stroke by clamping his tongue between his teeth, and letting up the pressure on the up-strokes. [See next page.]
    TARGET: Er nahm eine glänzend geschliffene Schindel auf, die im Mondlicht lag, zog ein Stückchen Rotstift aus der Tasche, ließ das Mondlicht sein Werk bescheinen, und kritzelte mühsam, jeden schwerfälligen Grundstrich hervorhebend, indem er die Zunge zwischen die Zähne klemmte und sie bei den Haarstrichen wieder freiließ, folgende Zeilen: ,,Huck Finn und Tom Sawyer schwöhren, Sie wolen 

# Checking the tensorboard files

In [4]:
import tensorboard as tb
import os

In [5]:
# If you are on ssh then make sure you are doing port forwarding
#ssh -L 6005:localhost:6005 user@jetson_ip
#Finally on your host on a browser open http://localhost:6005

logdir = "./runs/tmodel/"
os.system(f"tensorboard --logdir {logdir} --port 6005")



TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.20.0 at http://localhost:6005/ (Press CTRL+C to quit)


2